In [12]:
import pandas as pd
import numpy as np
import requests

INV_CSV_IN  = "coops_waterlevel_station_inventory.csv"
INV_CSV_OUT = "coops_waterlevel_station_inventory_fixed.csv"

BASE_MDAPI = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi"
BASE_DATA  = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

sess = requests.Session()
sess.headers.update({"Accept":"application/json"})

def get_json(url, params=None, timeout=60):
    r = sess.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json()

def parse_dt(x):
    if x is None or (isinstance(x, str) and x.strip() == ""):
        return pd.NaT
    return pd.to_datetime(x, errors="coerce", utc=True)

def extract_station_list(j):
    if isinstance(j, dict) and "stationList" in j:
        return j["stationList"]
    # fallback: first list-of-dicts value
    if isinstance(j, dict):
        for v in j.values():
            if isinstance(v, list) and len(v) and isinstance(v[0], dict):
                return v
    raise KeyError(f"No station list found. Keys={list(j.keys()) if isinstance(j, dict) else type(j)}")

def get_details(sid):
    return get_json(f"{BASE_MDAPI}/stations/{sid}/details.json")

def details_span(det, now_utc):
    d = det.get("details", det) if isinstance(det, dict) else {}
    established = parse_dt(d.get("established") or d.get("beginDate") or d.get("begin_date"))
    removed     = parse_dt(d.get("removed")     or d.get("endDate")   or d.get("end_date"))
    if pd.isna(established):
        return established, removed, np.nan
    end = removed if pd.notna(removed) else now_utc
    yrs = float((end - established).days / 365.25)
    return established, removed, yrs

def has_wind_data_api(sid):
    """
    Definitive check: does Data API return wind for a tiny window?
    If station doesn't support wind product, API returns an error message.
    """
    params = dict(
        product="wind",
        station=str(sid),
        begin_date="20200101",
        end_date="20200102",
        time_zone="gmt",
        units="metric",
        format="json",
    )
    r = requests.get(BASE_DATA, params=params, timeout=60)
    r.raise_for_status()
    j = r.json()
    # Wind success typically returns a dict containing 'data' list; errors contain 'error' keys/messages
    if isinstance(j, dict) and ("data" in j) and isinstance(j["data"], list):
        return True
    return False

# ------------------------------------------------------------
# A) Diagnose current CSV
# ------------------------------------------------------------
try:
    cur = pd.read_csv(INV_CSV_IN)
    print("Existing CSV loaded:", INV_CSV_IN, "rows=", len(cur))
    if "years_operation" in cur.columns:
        print("years_operation NaNs:", int(pd.to_numeric(cur["years_operation"], errors="coerce").isna().sum()))
    if "has_wind" in cur.columns:
        # accept strings like "True"/"False"
        hw = cur["has_wind"].astype(str).str.lower().isin(["true","1","yes"])
        print("has_wind True:", int(hw.sum()))
    print("Columns:", list(cur.columns))
except FileNotFoundError:
    print("Existing CSV not found (will rebuild from scratch).")
    cur = None

# ------------------------------------------------------------
# B) Rebuild: stations list + details (years) + wind via Data API
# ------------------------------------------------------------
j = get_json(f"{BASE_MDAPI}/stations.json", params={"type":"historicwl"})
stations = extract_station_list(j)

now_utc = pd.Timestamp.now(tz="UTC")
rows = []

for s in stations:
    sid = s.get("id")
    if not sid:
        continue

    row = {
        "id": sid,
        "name": s.get("name"),
        "state": s.get("state"),
        "lat": pd.to_numeric(s.get("lat"), errors="coerce"),
        "lon": pd.to_numeric(s.get("lng"), errors="coerce"),
    }

    # years of operation from details
    try:
        det = get_details(sid)
        established, removed, yrs = details_span(det, now_utc)
    except Exception as e:
        established, removed, yrs = (pd.NaT, pd.NaT, np.nan)
        row["details_error"] = str(e)[:160]

    row["established"] = established
    row["removed"] = removed
    row["years_operation"] = yrs

    # wind availability from Data API (definitive)
    try:
        row["has_wind"] = bool(has_wind_data_api(sid))
    except Exception as e:
        row["has_wind"] = False
        row["windcheck_error"] = str(e)[:160]

    rows.append(row)

tab = pd.DataFrame(rows).sort_values(["years_operation","id"], ascending=[False, True]).reset_index(drop=True)
tab.to_csv(INV_CSV_OUT, index=False)

print("\nRebuilt CSV:", INV_CSV_OUT)
print("Rows:", len(tab))
print("years_operation NaNs:", int(tab["years_operation"].isna().sum()))
print("has_wind True:", int(tab["has_wind"].sum()))

print("\nExample wind stations (first 20):")
print(tab.loc[tab["has_wind"], ["id","name","state","years_operation"]].head(20).to_string(index=False))


Existing CSV loaded: coops_waterlevel_station_inventory.csv rows= 2912
years_operation NaNs: 20
has_wind True: 1
Columns: ['id', 'name', 'state', 'lat', 'lon', 'established', 'removed', 'years_operation', 'has_wind', 'details_error']

Rebuilt CSV: coops_waterlevel_station_inventory_fixed.csv
Rows: 2912
years_operation NaNs: 20
has_wind True: 54

Example wind stations (first 20):
     id                       name                    state  years_operation
8665530                 Charleston                       SC       126.291581
8574680                  Baltimore                       MD       123.496235
1612340                   Honolulu                       HI       120.991102
8531680                 Sandy Hook                       NJ       115.975359
8557380                      Lewes                       DE       106.956879
8594900                 Washington                       DC       101.133470
8410140                   Eastport                       ME        96.295688
84

In [14]:
tab.query("state == 'TX' and years_operation >= 30") \
  .sort_values("years_operation", ascending=False)

,id,name,state,lat,lon,established,removed,years_operation,has_wind,windcheck_error,details_error
28,8771450,Galveston Pier 21,TX,29.310000,-94.793304,1904-01-01 00:00:00+00:00,NaT,121.993155,False,403 Client Error: Forbidden for url: https://a...,NaN
94,8774770,Rockport,TX,28.021667,-97.046670,1937-02-25 00:00:00+00:00,NaT,88.840520,False,403 Client Error: Forbidden for url: https://a...,NaN
123,8779770,Port Isabel,TX,26.061167,-97.215530,1944-03-31 00:00:00+00:00,NaT,81.746749,False,403 Client Error: Forbidden for url: https://a...,NaN
145,8771416,"Galveston Bay Entrance, South Jetty",TX,29.326700,-94.693300,1925-05-02 00:00:00+00:00,2003-07-15 23:59:00+00:00,78.201232,False,403 Client Error: Forbidden for url: https://a...,NaN
213,8778490,Port Mansfield,TX,26.557638,-97.425720,1962-08-11 00:00:00+00:00,NaT,63.383984,False,403 Client Error: Forbidden for url: https://a...,NaN
278,8771510,Galveston Pleasure Pier,TX,29.285300,-94.789400,1957-01-01 00:00:00+00:00,2011-07-20 23:59:00+00:00,54.546201,False,403 Client Error: Forbidden for url: https://a...,NaN
284,8772440,Freeport,TX,28.948300,-95.308300,1954-04-08 00:00:00+00:00,2008-03-19 23:59:00+00:00,53.946612,False,403 Client Error: Forbidden for url: https://a...,NaN
303,8771013,"Eagle Point, Galveston Bay",TX,29.481306,-94.917250,1973-06-14 00:00:00+00:00,NaT,52.542094,False,403 Client Error: Forbidden for url: https://a...,NaN
304,8770613,"Morgans Point, Barbours Cut",TX,29.681667,-94.985000,1973-06-18 00:00:00+00:00,NaT,52.531143,False,403 Client Error: Forbidden for url: https://a...,NaN
427,8772479,Freeport Entrance Jetty,TX,28.930300,-95.305900,1977-10-20 00:00:00+00:00,2020-06-15 00:01:00+00:00,42.652977,False,403 Client Error: Forbidden for url: https://a...,NaN
